In [3]:
import pandas as pd 
import numpy as np 
import wrds
import os
import requests
import unicodedata

In [4]:
db = wrds.Connection(wrds_username='codywan')

Loading library list...
Done


In [3]:
""" prepare for scraping corporate annoucments/articles/press on sustainability """

' prepare for scraping corporate annoucments/articles/press on sustainability '

In [4]:
# get permno_list, to get weburl of company for scraping
permno_list = pd.read_csv("data/permno/filtered_permno_list_19960105_20200101.csv", header=None, dtype='int').iloc[0].values
len(permno_list)

775

In [5]:
# get linking table from permno to gvkey
LINKING_FILE_PATH = '/Users/codywan/Data/WRDS Data/crspa_ccmlinktable.csv'
link = pd.read_csv(LINKING_FILE_PATH, header=0, keep_default_na=False).replace("", np.NaN).dropna(subset=['LPERMNO', 'GVKEY'])
link[['LPERMNO', 'GVKEY']] = link[['LPERMNO', 'GVKEY']].astype('int')
permno_gvkey_mapping = {int(permno): list(set(link[(link['LPERMNO']==permno)]['GVKEY'])) for permno in link['LPERMNO'].unique()}
# get gvkey_list from permno_list (to fetch company weburl data) 
gvkey_list = [permno_gvkey_mapping[permno] for permno in permno_list if permno in permno_gvkey_mapping]
# a few has no matching (e.g. 16267 ARROWSTREET CAPITAL, 
# which i'm very confused, i thought it was a private asset management firm) 
# see db.raw_sql("SELECT * FROM crsp.msf WHERE permno=16267")
# and a few has one permno matched to multiple gvkey:
print(len(gvkey_list), sum([len(gvkey) for gvkey in gvkey_list]))
# flatten gvkey_list to get company name and url from wrds
gvkey_list = [a for b in gvkey_list for a in b]
print(len(gvkey_list))

772 781
781


In [6]:
# conml: company legal name; weburl: website url
res = db.raw_sql(f"SELECT gvkey, conml, weburl FROM comp.company where CAST(gvkey AS INT) in ({','.join(str(g) for g in gvkey_list)})")
res = res.dropna(subset=['weburl'], axis=0, how='any').astype({'gvkey':'int64'}).set_index('gvkey')
res['mkvalt'] = [0]*len(res)
len(res), res.head()
# res.to_csv("data/web_scraping/seeds.txt", index=False)

(566,
                                   conml                   weburl  mkvalt
 gvkey                                                                   
 1072                          AVX Corp.              www.avx.com       0
 1414                      Primerica Inc        www.primerica.com       0
 1177                         Aetna Inc.            www.aetna.com       0
 1487   American International Group Inc              www.aig.com       0
 1447                American Express Co  www.americanexpress.com       0)

In [8]:
# df = db.raw_sql(f"SELECT gvkey, fyear, mkvalt FROM compa.funda "
#                 f"where CAST(gvkey AS INT) in ({','.join(str(g) for g in gvkey_list)})"
#                 f"AND mkvalt IS NOT NULL")
# df.to_csv("data/web_scraping/gvkey_mkvalt.csv")

# # read gvkey to mkvalt (market cap) from local
df = pd.read_csv("data/web_scraping/gvkey_mkvalt.csv", index_col=0, dtype={'gvkey':'int64'})

In [9]:
# map mkvalt to dataframe
for row in res.itertuples():
    gvkey = getattr(row, 'Index')
    df_gvkey = df[df['gvkey']==gvkey]
    if not df_gvkey.empty:
        res.at[gvkey, 'mkvalt']=df_gvkey.iloc[-1]['mkvalt']

In [10]:
res = res.sort_values('mkvalt', ascending=False)
res.reset_index().head()

,gvkey,conml,weburl,mkvalt
0,160329,Alphabet Inc,www.abc.xyz,729439
1,170617,Facebook Inc,investor.fb.com,512792
2,12141,Microsoft Corp,www.microsoft.com,354392
3,179534,Visa Inc,usa.visa.com,333174
4,4503,Exxon Mobil Corp,corporate.exxonmobil.com,323960


In [222]:
# shuffle rows of the dataframe
# for a different order of scraping websites
res = res.sample(frac=1)

In [224]:
# NOTE: don't run, unless prepping for scraping
# save company name, weburl to local files, to be read by a python script that does scraping
n = 14
count = 0
for i in range(len(res)):
    # if count < n * 5:
    #     count += 1
    #     continue
    file_num = i%n + 1
    with open(f"data/web_scraping/seeds_part{file_num}.txt", 'a') as f:
        conml, weburl = res.iloc[i][['conml', 'weburl']]
        if not weburl == 'www.americanexpress.com':
            f.write(f"{conml},{weburl}\n")

In [11]:
""" test a few functionalities used for scraping """

' test a few functionalities used for scraping '

In [ ]:
# NOTE: test dragnet, for extracting meaningful content of a web page

from dragnet import extract_content
# url = r"https://www.microsoft.com/en-us/corporate-responsibility/sustainability"
url = r"https://www.microsoft.com/en-us/corporate-responsibility/sustainability/operations"
response = requests.get(url)
content = extract_content(response.content)

In [ ]:
unicodedata.normalize("NFKD", content).replace("\n", " ").replace("\t", "").replace("  ", " ")

In [ ]:
# dragnet is pretty accurate at getting the content but it also misses a lot of things compared to 
# good old-fashioned soup.find_all('p') using the same url
from bs4 import BeautifulSoup
soup = BeautifulSoup(requests.get(url).content, 'html5')
text = soup.find_all('p')
for t in text:
    t = t.get_text()
    page_text_norm = str(unicodedata.normalize('NFKD', t).encode(encoding='ascii', errors='ignore'))
    # print(page_text_norm)

In [ ]:
# however, we just need to identify some texts on sustainability with high confidence. Let's use dragnet on
# every company in our list and see if collectively, their word distribution skew towards sustainability, csr, etc.

In [ ]:
# NOTE: test whitelist and blacklist words

from scrapy.spiders import Spider
from scrapy.http import Request
from scrapy.http.response.html import HtmlResponse
from scrapy.linkextractors import LinkExtractor
from urllib.parse import urlparse
import re

In [ ]:
url = r"https://www.microsoft.com"
company = "microsoft"
request = Request(url)
request.meta.update(company=company.strip())

In [ ]:
whitelist = ['csr', 'esg', 'responsibility', 'sustainability', 'climate', 'conservation'
                          'environment', 'renewable', 'carbon-footprint', 'clean-energy', 'carbon-neutral',
                          'corporate-governance', 'diverse-workforce', 'climate-change', 'customer-privacy',
                          'community-impact', 'clean-water']
blacklist = ['document', 'blog', 'product', 'profit', 'revenue', 'archive', 'search', 'login',
                          'accessories', 'shop', 'support', 'developer', 'dmg', 'forums']

In [ ]:
link_str = ' https://support.apple.com/es-us/guide/iphone/iph3dd5fc7e/12.0/ios/12.0'
# keywords stores matching of any word from whitelist (| means OR), re.I means ignore case
keywords = list(set(re.findall("|".join(whitelist), link_str, flags=re.I)))
flashcards = list(set(re.findall("|".join(blacklist), link_str, flags=re.I)))

In [12]:
""" after scraping, process scraped content stored in local mongodb database """

' process scraped content stored in local mongodb database '

In [31]:
from pymongo import MongoClient

class Connect(object):
    @staticmethod    
    def get_connection():
        return MongoClient(port=27017, username='codywan', password='password', authSource="admin")

# connect to local mongodb
connection = Connect.get_connection()
db = connection['admin']

In [32]:
# get whitelist used in scraping, for compiling statistics
keywords_dict = dict()

# further process scraped content

from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)

def convert_unicode(content):
    return unicodedata.normalize("NFKD", content).replace("\n", " ").replace("\t", "").replace("  ", " ")

filter_list = ["system.users", 'system.version']
master_texts = list()
for collection_name in db.collection_names():
    if collection_name not in filter_list:
        # only store unique texts/str
        sub_texts = set()
        cursor = db[collection_name].find({})
        for item in cursor:
            # convert any leftover unicode
            text = convert_unicode(item['content'])
            # skip non-English texts
            lang, prob = identifier.classify(text)
            if prob > 0.8 and lang == 'en':
                sub_texts.add(text)
            # get count for keywords
            for keyword in item['keywords']:
                keywords_dict[keyword] = keywords_dict.get(keyword, 0) + 1
        master_texts.extend(list(sub_texts))

In [34]:
keywords_dict

{'sustainability': 46875,
 'carbon-neutral': 114,
 'corporate-responsibility': 425,
 'climate-change': 749,
 'corporate-governance': 140,
 'social-responsibility': 252,
 'community-impact': 36,
 'clean-energy': 203,
 'global-warming': 26,
 'clean-water': 65,
 'social-justice': 14,
 'social-impact': 98,
 'carbon-footprint': 34,
 'inclusive-environment': 4,
 'customer-privacy': 3,
 'diverse-workforce': 2}

In [37]:
# save processed text corpora to local
import pickle
with open("data/web_scraping/sustainability_corpora.pickle", "wb") as f:
    pickle.dump(master_texts, f)

In [38]:
""" after saving processed scraped content to local as corpora, prepare corpora for LDA """

' after saving processed scraped content to local as corpora, prepare corpora for LDA '

In [1]:
import pickle
# load text corpora
with open("data/web_scraping/sustainability_corpora.pickle", "rb") as f:
    texts = pickle.load(f)

In [2]:
len(texts)

15407

In [41]:
# convert text to bag of words:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

wordlemmatizer = WordNetLemmatizer()
list_of_words = list()

for text in texts:
    # tokenize text as English
    text_tokenized = word_tokenize(text, language='English')
    # convert to lowercases and remove puncutation
    text_tokenized = [word.lower() for word in text_tokenized if word.isalpha()]
    # lemmatize token
    text_tokenized = [wordlemmatizer.lemmatize(word) for word in text_tokenized if word not in stopwords.words('english')]
    list_of_words.append(text_tokenized)

In [42]:
# check content with very small number of words (likely off-topic content)
len([words for words in list_of_words if len(words) <= 20])

81

In [43]:
# remove texts with few words (likely scraped from comment or survey section with no substantial information)
list_of_words = [words for words in list_of_words if len(words) > 20]

In [44]:
# save processed text tokenized to local
with open("data/web_scraping/sustainability_corpora_tokenized.pickle", "wb") as f:
    pickle.dump(list_of_words, f)

In [45]:
""" after saving tokenized corpora to local, load and prepare tokenized corpora for LDA """

' after saving tokenized corpora to local, load and prepare tokenized corpora for LDA '

In [1]:
import pickle
with open("data/web_scraping/sustainability_corpora_tokenized.pickle", "rb") as f:
    list_of_words = pickle.load(f)

In [2]:
len(list_of_words)

15326

In [3]:
# NOTE: run LDA
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim import corpora, models
import gensim
import pyLDAvis.gensim

# create dictionary based on list of words
dictionary = Dictionary(list_of_words)
# remove tokens appear less than no_below and more than no_above (in %)
dictionary.filter_extremes(no_above=0.8)
bow_corpus = [dictionary.doc2bow(doc) for doc in list_of_words]

In [5]:
# apply tf-idf
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [6]:
# fit with bag of words (without tf-idf)
num_topics=6
path=f"data/LDA/lda_model_{num_topics}topics"
if not os.path.exists(path):
    os.mkdir(path)
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=num_topics, id2word=dictionary, workers=7)
# save model specifications to local
lda_model.save(f"{path}/topic_model.model")
dictionary.save(f"{path}/dictionary.dict")
corpora.MmCorpus.serialize(f"{path}/corpus.mm", bow_corpus)
# visualize LDA
res = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.save_html(res,f'{path}/vis.html')

In [21]:
# show distribution of words for each topic (-1 choose all topics), ordered by significance
# for idx, topic in lda_model.print_topics(-1):
#     print(f'Topic: {idx} \nWords: {topic}')

In [22]:
# fit with bag of words with tf-idf
path="data/LDA/lda_model_tfidf"
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, workers=7)
# save model specifications to local
lda_model.save(f"{path}/topic_model.model")
dictionary.save(f"{path}/dictionary.dict")
corpora.MmCorpus.serialize(f"{path}/corpus.mm", bow_corpus)
# visualize LDA
res = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.save_html(res,f'{path}/vis.html')

In [ ]:
# for idx, topic in lda_model_tfidf.print_topics(-1):
#     print(f'Topic: {idx} \nWords: {topic}')

In [99]:
# performance evaluation by classifying sample document using lda bag of words model
for index, score in sorted(lda_model[dictionary.doc2bow(list_of_words[2])], key=lambda topic: -1*topic[1]):
    print(f"\nScore: {score}\t \nTopic: {lda_model.print_topic(index, 10)}")


Score: 0.9526092410087585	 
Topic: 0.013*"company" + 0.011*"energy" + 0.009*"climate" + 0.006*"sustainability" + 0.006*"said" + 0.006*"change" + 0.006*"business" + 0.005*"community" + 0.005*"environmental" + 0.005*"report"


In [24]:
""" load fitted lda model from local """

path="data/LDA/lda_model"
lda_model = gensim.models.LdaModel.load(f'{path}/topic_model.model')
dictionary = gensim.corpora.Dictionary.load(f'{path}/dictionary.dict')
bow_corpus = gensim.corpora.MmCorpus(f'{path}/corpus.mm')

In [35]:
# show distribution of words for each topic (-1 choose all topics), ordered by significance
# for idx, topic in lda_model.print_topics(-1):
#     print(f'Topic: {idx+1} \nWords: {topic}')

In [46]:
# apply LDA model to signal filings from keywords matching
# !python utils_LDA.py

In [ ]:
""" evaluate LDA signal """

In [13]:
import importlib
import wrds
import numpy as np 
import pandas as pd
import json
import utils.backtest_performance
importlib.reload(utils.backtest_performance)
from utils.backtest_performance import price_history_to_cumu, add_to_return_stats, get_price_history_yahoo_finance, get_price_history, get_return_stats, get_NACIS_code_sector_name

db = wrds.Connection(wrds_username='codywan')
# db.create_pgpass_file() # first-time use
def get_CIK_mapping():
    """
    """
    # get linking table from CIK to permno (for retreiving price history from wrds)
    LINKING_FILE_PATH = '/Users/codywan/Data/WRDS Data/crspa_ccmlinktable.csv'
    link = pd.read_csv(LINKING_FILE_PATH, header=0, keep_default_na=False).replace("", np.NaN).dropna(subset=['LPERMNO', 'cik'])
    link[['LPERMNO', 'cik']] = link[['LPERMNO', 'cik']].astype('int')
    CIK_PERMNO_mapping = {CIK: set(link[(link['cik']==CIK)]['LPERMNO']) for CIK in link['cik'].unique()}

    # get linking table from CIK to ticker (for retreiving price history after 12/31/2019 from yahoo finance)
    link['tic'] = link['tic'].astype('str')
    CIK_TICKER_mapping = {CIK: list(set(link[(link['cik']==CIK)]['tic'])) for CIK in link['cik'].unique()}

    # check if the mapping between CIK and TICKER is one-to-one
    if len(CIK_TICKER_mapping) == sum([len(CIK_TICKER_mapping[CIK]) for CIK in CIK_TICKER_mapping]):
        CIK_TICKER_mapping = {CIK: CIK_TICKER_mapping[CIK][0] for CIK in CIK_TICKER_mapping}
    else:
        print("multiple tickers for CIK")

    # get linking table from CIK to SIC (for benchmarking using industry level returns from  ken french website)
    CIK_SIC_mapping = {CIK: int(link[(link['cik']==CIK)]['sic'].unique().tolist()[0]) for CIK in link['cik'].unique()}
    return CIK_PERMNO_mapping, CIK_TICKER_mapping, CIK_SIC_mapping

Loading library list...
Done


In [14]:
# read SIC to industry portfolio mapping
with open("data/industry_classification_and_portfolio/SIC_portfl_mapping_Siccodes49.json", "r") as f:
    SIC_portfolio_mapping = json.load(f)

In [15]:
CIK_PERMNO_mapping, CIK_TICKER_mapping, CIK_SIC_mapping = get_CIK_mapping()

In [16]:
NACIS_sector_code, NACIS_sector_name = get_NACIS_code_sector_name()

In [97]:
import logging
# my_logger = logging.getLogger()
# my_logger.disabled = True

logging.basicConfig(filename="logs/generate_price_history/log.txt",
                    filemode='a',
                    level=logging.INFO,
                    format='%(levelname)s: %(asctime)s - %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S')

                    
repository_path = "backtests/LDA"

# get price history and performance metrics for each signal event
HOLDING_PERIOD = 30*5 # in days

master_df = list()
for sector_name in NACIS_sector_name:
    
    file_path = f"{repository_path}/{sector_name}/signal.csv"
    if not Path(file_path).is_file():
        continue
    print(sector_name)

    # get signal events
    df = pd.read_csv(file_path)
    BUY_SIGNAL = { int(col): df[col].dropna().to_list() for col in df}

    # get price history from wrds
    price_history, volume_on_event_day, no_price_history_signal = get_price_history(db, CIK_PERMNO_mapping, BUY_SIGNAL, HOLDING_PERIOD)
    # get price history of those not available from wrds, from yahoo finance
    ex_price_history, ex_volume_on_event_day, _ = get_price_history_yahoo_finance(CIK_TICKER_mapping, no_price_history_signal, HOLDING_PERIOD)
    # put price history from wrds and yahoo finance together
    price_history[ex_price_history.columns] = ex_price_history
    volume_on_event_day = pd.concat([volume_on_event_day, ex_volume_on_event_day])

    # compute return statistics
    return_stats = get_return_stats(db, price_history, CIK_SIC_mapping, SIC_portfolio_mapping)

    # add volume data to return stats
    add_to_return_stats(index=3, df=volume_on_event_day, return_stats=return_stats)

    # get earnings call flag and add to return stats
    earnings_call_date, _ = utils.backtest_performance.get_earnings_call_date(db, CIK_TICKER_mapping, BUY_SIGNAL)
    add_to_return_stats(index=4, df=earnings_call_date, return_stats=return_stats)

    # append to master variables
    master_df.append(return_stats)
    display(return_stats.sort_values("R", ascending=False).iloc[:5])
    # save to local
    return_stats.to_csv(f"{repository_path}/{sector_name}/performance.csv")

    # break

# sector-aggregated performance table to local
pd.concat(master_df).to_csv(f"{repository_path}/performance.csv")

Agriculture, Forestry, Fishing and Hunting
8/8 price history generated from wrds/crsp
0/0 price history generated from yahoo finance
8/8 earnings call date checked


R       MKT   R - MKT       vol   EA  \
CIK     day -1     day 0                                                     
18169   2012-09-17 2012-09-18  1.015328  1.000297  0.015031  4.063723  NaN   
1110783 2008-03-31 2008-04-01  1.004484  1.032770 -0.028285  1.191451  1.0   
        2013-01-30 2013-01-31  0.995522  1.009092 -0.013570  0.769629  NaN   
18169   2012-09-20 2012-09-21  0.994425  0.997799 -0.003374  2.400726  NaN   
1110783 2017-01-26 2017-01-27  0.988798  0.992008 -0.003210  0.722790  NaN   

                               10d - MKT  30d - MKT  60d - MKT  90d - MKT  \
CIK     day -1     day 0                                                    
18169   2012-09-17 2012-09-18   0.023594  -0.060028  -0.109767  -0.219718   
1110783 2008-03-31 2008-04-01   0.090976   0.017837   0.171132  -0.004116   
        2013-01-30 2013-01-31  -0.024049  -0.031472  -0.029140  -0.078926   
18169   2012-09-20 2012-09-21  -0.066822  -0.141302  -0.193622  -0.273133   
1110783 2017-01-26 2017-01-27  -0.029457   0.007649   0.022494   0.010466   

                               120d - MKT    IDT  10d - IDT  30d - IDT  \
CIK     day -1     day 0                                                 
18169   2012-09-17 2012-09-18   -0.302071  Agric  -0.008352  -0.060351   
1110783 2008-03-31 2008-04-01    0.096612  Agric   0.006831  -0.004927   
        2013-01-30 2013-01-31   -0.153300  Agric  -0.001248  -0.000581   
18169   2012-09-20 2012-09-21   -0.328859  Agric  -0.069676  -0.138451   
1110783 2017-01-26 2017-01-27   -0.016598  Agric  -0.000824   0.010097   

                               60d - IDT  90d - IDT  120d - IDT  
CIK     day -1     day 0                                         
18169   2012-09-17 2012-09-18  -0.138513  -0.324356   -0.372845  
1110783 2008-03-31 2008-04-01   0.015511  -0.010566   -0.004276  
        2013-01-30 2013-01-31  -0.004012  -0.014729   -0.026945  
18169   2012-09-20 2012-09-21  -0.207239  -0.338833   -0.365333  
1110783 2017-01-26 2017-01-27   0.007999   0.010048    0.005113

Mining, Quarrying, and Oil and Gas Extraction
212/258 price history generated from wrds/crsp
9/11 price history generated from yahoo finance
245/258 earnings call date checked


,,,R,MKT,R - MKT,vol,EA,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
CIK,day -1,day 0,,,,,,,,,,,,,,,,
773910,2014-04-02,2014-04-03,1.168613,0.982246,0.186368,0.851952,NaN,0.164832,0.164451,0.232864,0.210824,0.162857,Oil,0.110630,0.108990,0.164560,0.173723,0.192149
1518832,2019-12-13,2019-12-16,1.163153,1.008407,0.154746,1.970733,NaN,0.240296,0.072927,-0.433110,-0.471730,-0.316202,Oil,0.221876,0.165123,-0.130052,-0.178056,-0.080214
1070412,2016-02-26,2016-02-29,1.154337,1.016339,0.137998,1.112579,NaN,0.475397,0.661811,0.705788,0.944704,1.194114,Oil,0.430704,0.589540,0.605133,0.844409,1.099941
831259,2016-11-08,2016-11-09,1.141439,1.017847,0.123592,0.780734,NaN,0.295063,0.070498,0.196267,-0.047473,-0.168796,Mines,0.174731,0.019104,0.074487,-0.036726,-0.109522
1524741,2015-10-26,2015-10-27,1.131396,1.010040,0.121356,0.618072,0.0,0.252973,0.273716,0.125774,0.251364,0.577935,Mines,0.294259,0.387221,0.295162,0.277310,0.519280


Utilities
223/355 price history generated from wrds/crsp
11/12 price history generated from yahoo finance
332/355 earnings call date checked


...

,,,R,MKT,R - MKT,vol,EA,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
CIK,day -1,day 0,,,,,,,,,,,,,,,,
1320414,2017-10-20,2017-10-23,1.071348,0.997491,0.073857,1.483208,NaN,-0.049977,-0.040646,-0.114386,-0.055194,-0.031206,Hlth,0.014671,0.006044,-0.100912,-0.102114,-0.060510
1045829,2011-11-14,2011-11-15,1.061111,0.989108,0.072003,0.766931,NaN,0.140153,0.124289,0.049340,-0.023483,-0.141217,Hlth,0.158227,0.174692,0.091286,0.046946,-0.092784
1437578,2013-04-10,2013-04-11,1.017148,1.000690,0.016458,0.358566,NaN,-0.005626,-0.012806,-0.021312,0.016360,-0.016514,PerSv,0.003261,-0.040289,-0.061149,0.014690,-0.005359
860730,2019-01-16,2019-01-17,1.016854,1.020497,-0.003642,0.793586,NaN,0.022133,-0.040774,-0.146461,-0.135743,-0.087984,Hlth,0.020493,-0.005419,-0.053147,-0.048448,-0.032019
1320414,2016-09-27,2016-09-28,1.005327,0.995745,0.009582,1.100299,NaN,0.056548,-0.126088,-0.033499,-0.129750,-0.176082,Hlth,0.063584,-0.029172,0.059280,-0.052323,-0.092782


Arts, Entertainment, and Recreation
24/31 price history generated from wrds/crsp
1/1 price history generated from yahoo finance
31/31 earnings call date checked


R       MKT   R - MKT       vol   EA  \
CIK     day -1     day 0                                                     
701374  2011-12-19 2011-12-20  1.080043  1.033062  0.046981  0.673755  NaN   
        2010-09-10 2010-09-13  1.038945  1.012392  0.026553  0.591688  NaN   
1564902 2015-08-05 2015-08-06  1.037931  0.987632  0.050299  1.530385  0.0   
1656239 2017-12-19 2017-12-20  1.030711  1.002500  0.028211  0.956442  NaN   
701374  2010-08-17 2010-08-18  1.024483  0.984663  0.039819  2.361030  NaN   

                               10d - MKT  30d - MKT  60d - MKT  90d - MKT  \
CIK     day -1     day 0                                                    
701374  2011-12-19 2011-12-20   0.028587   0.045598   0.066155   0.088994   
        2010-09-10 2010-09-13   0.052957   0.192830   0.224000   0.201989   
1564902 2015-08-05 2015-08-06   0.112555   0.079425   0.163290   0.084794   
1656239 2017-12-19 2017-12-20  -0.008109  -0.003761  -0.066473   0.021434   
701374  2010-08-17 2010-08-18   0.147650   0.227516   0.406509   0.439672   

                               120d - MKT  IDT  10d - IDT  30d - IDT  \
CIK     day -1     day 0                                               
701374  2011-12-19 2011-12-20    0.235092  Fun   0.009532  -0.074887   
        2010-09-10 2010-09-13    0.326795  Fun   0.035403   0.129236   
1564902 2015-08-05 2015-08-06    0.209264  Fun   0.164825   0.152916   
1656239 2017-12-19 2017-12-20   -0.011662  Fun  -0.011698  -0.141719   
701374  2010-08-17 2010-08-18    0.539067  Fun   0.166571   0.199279   

                               60d - IDT  90d - IDT  120d - IDT  
CIK     day -1     day 0                                         
701374  2011-12-19 2011-12-20  -0.104348  -0.026459    0.241854  
        2010-09-10 2010-09-13   0.049928   0.064094    0.212719  
1564902 2015-08-05 2015-08-06   0.221021   0.152378    0.289041  
1656239 2017-12-19 2017-12-20  -0.306346  -0.235856   -0.398724  
701374  2010-08-17 2010-08-18   0.187701   0.294820    0.371449

Accommodation and Food Services
190/227 price history generated from wrds/crsp
9/12 price history generated from yahoo finance
217/227 earnings call date checked


R       MKT   R - MKT       vol  EA  \
CIK    day -1     day 0                                                    
858339 2014-11-11 2014-11-12  1.482047  1.000400  0.481647  0.766720 NaN   
       2013-04-22 2013-04-23  1.315452  1.011707  0.303746  2.324882 NaN   
       2016-07-29 2016-08-01  1.150725  0.991411  0.159313  0.498378 NaN   
       2014-12-18 2014-12-19  1.138952  1.008016  0.130936  2.086160 NaN   
       2014-12-19 2014-12-22  1.129726  1.005808  0.123918  1.412747 NaN   

                              10d - MKT  30d - MKT  60d - MKT  90d - MKT  \
CIK    day -1     day 0                                                    
858339 2014-11-11 2014-11-12   0.520094   0.361173  -0.047886  -0.181175   
       2013-04-22 2013-04-23   0.185665   0.022828   0.223311   0.674142   
       2016-07-29 2016-08-01   0.068352  -0.025810   0.055750   0.131462   
       2014-12-18 2014-12-19   0.038639  -0.137135  -0.311356  -0.327851   
       2014-12-19 2014-12-22  -0.000844  -0.147795  -0.332249  -0.344986   

                              120d - MKT  IDT  10d - IDT  30d - IDT  \
CIK    day -1     day 0                                               
858339 2014-11-11 2014-11-12   -0.141596  Fun   0.536483   0.436060   
       2013-04-22 2013-04-23    0.635222  Fun   0.143411  -0.023512   
       2016-07-29 2016-08-01    0.252755  Fun   0.049604  -0.087516   
       2014-12-18 2014-12-19   -0.510855  Fun   0.026276  -0.208316   
       2014-12-19 2014-12-22   -0.544683  Fun  -0.001818  -0.193420   

                              60d - IDT  90d - IDT  120d - IDT  
CIK    day -1     day 0                                         
858339 2014-11-11 2014-11-12  -0.022000  -0.136845   -0.112960  
       2013-04-22 2013-04-23   0.188519   0.591865    0.462559  
       2016-07-29 2016-08-01  -0.100688  -0.029959    0.110385  
       2014-12-18 2014-12-19  -0.373273  -0.413537   -0.631239  
       2014-12-19 2014-12-22  -0.366452  -0.410255   -0.641130

In [9]:
# by time periods and remove earnings call

from scipy.stats import ttest_1samp
import pandas as pd 
import numpy as np
from pathlib import Path

from utils.backtest_performance import price_history_to_cumu, add_to_return_stats, get_price_history_yahoo_finance, get_price_history, get_return_stats, get_NAICS_code_sector_name
# generate pair of NAICS sector code and name

NAICS_sector_code, NAICS_sector_name = get_NAICS_code_sector_name()


vol_threshold = 1.25
COLUMNS = ['1d - MKT', '10d - MKT', '30d - MKT', '60d - MKT', '90d - MKT', '120d - MKT','1d - IDT', '10d - IDT', '30d - IDT', '60d - IDT', '90d - IDT', '120d - IDT']
# TIME_PARTITION = [['1995-01-01', '1999-12-31'], ['2000-01-01', '2004-12-31'], 
#                 ['2005-01-01', '2009-12-31'], ['2010-01-01', '2014-12-31'], 
#                 ['2015-01-01', '2019-12-31']]

TIME_PARTITION = [['1995-01-01', '1999-12-31'], ['2000-01-01', '2005-12-31'], 
                ['2006-01-01', '2007-12-31'], ['2008-01-01', '2009-12-31'],
                ['2010-01-01', '2011-12-31'], ['2012-01-01', '2013-12-31'], 
                ['2014-01-01', '2015-12-31'], ['2016-01-01', '2017-12-31'],
                ['2018-01-01', '2019-12-31']]

repository_path = f"backtests/LDA/ESG_baseline"
master_df = list()
for (t0_str, t1_str) in TIME_PARTITION[1:]:
    t0, t1 = pd.to_datetime(t0_str), pd.to_datetime(t1_str)
    print(f"{t0_str} - {t1_str}")
    master_t_stats = list()

    for i, sector_name in enumerate(NAICS_sector_name):

        file_path = f"{repository_path}/{sector_name}/performance.csv"
        if not Path(file_path).is_file():
            continue

        # get performance df for this sector
        df = pd.read_csv(file_path, parse_dates=[2])
        # select those in t0 to t1 periods
        df = df[(t0 < df['day 0']) & (df['day 0'] <= t1)]
        # select a subset, that tries to remove earnings call, merger etc. events
        df = df[(df['EA'].isnull()) & (df['vol'] <= vol_threshold)]

        if len(df) == 0:
            continue

        # compute t-stats
        t_stats = df[COLUMNS].apply(lambda col: f"{col.mean():.4f} ({ttest_1samp(list(col), popmean=0)[1]*100:.2f}%)", axis=0).to_frame().T
        t_stats.insert(0, 'sample size', len(df))
        t_stats.index = [f"{i}"]
        master_t_stats.append(t_stats)

    # if there's signal events in this time period
    if len(master_t_stats) != 0:
        # compute performance for all signal events in this basket
        df = pd.read_csv(f"{repository_path}/performance.csv", parse_dates=[2])
        df = df[(t0 < df['day 0']) & (df['day 0'] <= t1)]
        df = df[(df['EA'].isnull()) & (df['vol'] <= vol_threshold)][COLUMNS]
        df = df.apply(lambda col: f"{np.round(col.mean(), 4)} ({np.round(ttest_1samp(list(col), popmean=0)[1]*100, 2)}%)", axis=0).to_frame().T
        df.index = [f"Total"]
        # add to master variable
        master_t_stats.append(df)
        all_sector_returns = pd.concat(master_t_stats)
        all_sector_returns.at['Total', 'sample size'] = all_sector_returns.iloc[:-1]['sample size'].sum()
        all_sector_returns.to_csv(f"{repository_path}/performance_evaluation_{t0_str}_{t1_str}.csv")
        # display to console
        display(all_sector_returns)
    else:
        print(f"{len(master_t_stats)} events from {t0_str} - {t1_str}")
    
    # break


2000-01-01 - 2005-12-31


,sample size,1d - MKT,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,1d - IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
2,1.0,0.0236 (nan%),-0.0246 (nan%),-0.0705 (nan%),-0.0988 (nan%),-0.0422 (nan%),-0.0171 (nan%),0.0121 (nan%),-0.0602 (nan%),-0.1108 (nan%),-0.1840 (nan%),-0.1311 (nan%),-0.1714 (nan%)
4,23.0,0.0097 (0.43%),0.0104 (39.52%),0.0242 (31.84%),0.0229 (44.63%),0.1095 (1.02%),0.1717 (0.03%),0.0063 (3.06%),0.0086 (44.87%),0.0296 (24.53%),0.0195 (53.43%),0.0886 (3.25%),0.1343 (0.26%)
5,2.0,0.0041 (90.50%),0.0041 (79.58%),-0.0321 (10.80%),-0.0511 (9.27%),-0.0439 (51.20%),-0.0246 (72.55%),0.0028 (92.40%),0.0120 (53.68%),-0.0283 (10.40%),-0.0591 (12.93%),-0.0511 (43.23%),-0.0407 (44.85%)
6,2.0,-0.0100 (9.33%),0.0385 (35.65%),0.0668 (34.94%),0.0486 (24.43%),0.1311 (2.83%),0.0486 (45.02%),-0.0085 (2.13%),0.0410 (48.84%),0.0592 (56.75%),0.0407 (27.36%),0.1559 (3.65%),0.0471 (54.26%)
7,3.0,0.0143 (47.81%),0.0014 (92.47%),0.0269 (61.48%),0.1169 (18.41%),0.0518 (26.69%),0.0777 (39.14%),0.0060 (66.79%),-0.0119 (67.72%),0.0055 (92.35%),0.0387 (57.09%),-0.0191 (61.73%),-0.0395 (47.35%)
8,12.0,-0.0094 (65.59%),-0.0195 (32.56%),0.0029 (92.59%),0.0464 (28.08%),0.0663 (40.13%),0.0014 (98.35%),-0.0114 (60.82%),-0.0304 (20.69%),-0.0150 (68.09%),0.0149 (62.12%),0.0620 (35.15%),0.0160 (81.25%)
9,17.0,0.0043 (57.95%),0.0195 (2.68%),0.0296 (12.69%),0.0693 (9.58%),0.1111 (6.96%),0.1356 (6.80%),0.0021 (72.60%),0.0220 (3.10%),0.0280 (11.81%),0.0616 (15.17%),0.0819 (17.41%),0.0976 (17.68%)
11,2.0,0.0059 (74.12%),0.0044 (91.65%),-0.0075 (97.38%),-0.0077 (97.81%),0.1389 (68.28%),0.3461 (51.78%),0.0069 (78.14%),0.0065 (83.33%),0.0067 (97.46%),-0.0250 (92.23%),0.1158 (68.84%),0.3249 (52.06%)
13,1.0,-0.0127 (nan%),0.0611 (nan%),-0.0373 (nan%),-0.0070 (nan%),0.0664 (nan%),0.1412 (nan%),-0.0030 (nan%),0.0598 (nan%),-0.0383 (nan%),-0.0210 (nan%),0.0260 (nan%),0.1091 (nan%)
17,5.0,0.0202 (32.15%),0.0124 (31.61%),0.0304 (37.64%),0.0475 (41.49%),0.0437 (40.87%),0.0629 (43.93%),0.0213 (35.96%),0.0147 (20.40%),0.0265 (35.58%),0.0404 (30.25%),0.0438 (30.79%),0.0524 (42.89%)


2006-01-01 - 2007-12-31


,sample size,1d - MKT,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,1d - IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
1,3.0,0.0262 (27.84%),0.0222 (64.14%),0.0540 (60.80%),0.0259 (42.95%),-0.0250 (87.11%),0.0783 (63.56%),0.0192 (27.90%),0.0131 (60.87%),0.0290 (64.07%),-0.0474 (34.07%),-0.0902 (39.39%),-0.0162 (87.27%)
2,6.0,0.0045 (68.70%),0.0067 (78.85%),0.0290 (29.39%),0.1147 (11.21%),0.1887 (10.87%),0.3272 (4.14%),0.0040 (69.21%),0.0071 (78.09%),0.0198 (40.75%),0.0602 (34.16%),0.1130 (26.76%),0.2265 (13.46%)
3,1.0,0.0200 (nan%),0.0581 (nan%),-0.1092 (nan%),0.0283 (nan%),0.1249 (nan%),0.3997 (nan%),0.0058 (nan%),0.1137 (nan%),-0.1041 (nan%),0.0301 (nan%),0.0925 (nan%),0.3217 (nan%)
4,35.0,0.0166 (8.48%),0.0255 (14.75%),0.0333 (14.48%),0.0729 (1.94%),0.0994 (1.47%),0.1291 (0.29%),0.0128 (18.25%),0.0185 (25.98%),0.0105 (62.60%),0.0301 (28.46%),0.0425 (23.49%),0.0489 (18.07%)
5,3.0,0.0091 (59.57%),-0.0101 (56.01%),-0.0337 (58.89%),-0.0384 (55.53%),-0.0240 (73.78%),-0.0312 (77.38%),0.0099 (57.63%),-0.0052 (83.37%),-0.0337 (61.18%),-0.0366 (51.88%),-0.0027 (97.22%),-0.0175 (87.49%)
6,5.0,-0.0059 (39.46%),0.0164 (33.71%),0.0794 (29.85%),0.0808 (15.23%),0.0571 (45.10%),0.0204 (75.98%),-0.0108 (11.62%),0.0215 (40.04%),0.0923 (21.29%),0.1086 (7.17%),0.1065 (20.31%),0.0838 (15.96%)
7,8.0,-0.0019 (88.47%),0.0009 (96.23%),0.0418 (7.94%),0.0887 (1.69%),0.1495 (2.71%),0.2059 (1.63%),0.0040 (69.89%),0.0085 (62.83%),0.0486 (4.44%),0.0849 (1.11%),0.1217 (3.51%),0.1490 (4.30%)
8,24.0,0.0022 (74.50%),-0.0103 (49.42%),-0.0120 (47.19%),-0.0228 (33.79%),-0.0200 (56.53%),-0.0130 (76.92%),0.0001 (98.77%),-0.0150 (26.82%),-0.0192 (20.23%),-0.0423 (4.68%),-0.0540 (9.77%),-0.0615 (11.94%)
9,14.0,0.0073 (16.63%),-0.0066 (72.76%),-0.0104 (67.32%),-0.0110 (77.68%),-0.0339 (39.67%),-0.0409 (34.12%),0.0012 (79.64%),-0.0101 (50.46%),-0.0141 (46.30%),-0.0031 (91.75%),-0.0266 (38.55%),-0.0406 (24.11%)
10,2.0,0.0071 (64.35%),-0.0011 (98.58%),0.0186 (44.50%),0.0442 (24.98%),0.0418 (70.69%),-0.2083 (25.38%),0.0015 (57.00%),-0.0053 (90.17%),0.0062 (13.97%),0.0201 (30.80%),0.0827 (29.39%),-0.0301 (62.96%)


2008-01-01 - 2009-12-31


...

,sample size,1d - MKT,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,1d - IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
1,33.0,-0.0100 (30.85%),-0.0316 (3.30%),-0.1002 (0.15%),-0.1916 (0.01%),-0.1860 (0.03%),-0.3067 (0.00%),-0.0103 (24.46%),-0.0250 (6.31%),-0.0723 (0.27%),-0.1179 (0.07%),-0.0906 (1.73%),-0.1739 (0.01%)
2,19.0,-0.0026 (59.54%),-0.0234 (2.63%),-0.0327 (10.54%),-0.0464 (13.96%),-0.0452 (22.75%),-0.0433 (24.71%),-0.0007 (87.58%),-0.0184 (8.70%),-0.0191 (19.53%),-0.0317 (28.31%),-0.0245 (49.98%),-0.0259 (49.84%)
3,2.0,0.0209 (1.82%),-0.0135 (28.04%),-0.0142 (74.84%),-0.0059 (93.21%),-0.0536 (25.02%),-0.0935 (23.53%),0.0190 (9.58%),-0.0373 (16.85%),-0.0104 (33.66%),0.0039 (5.53%),-0.0278 (31.97%),-0.0319 (57.41%)
4,149.0,0.0024 (30.91%),0.0046 (43.52%),-0.0040 (63.56%),-0.0142 (18.46%),-0.0259 (2.33%),-0.0215 (13.68%),0.0014 (53.68%),0.0033 (57.01%),-0.0089 (28.95%),-0.0168 (12.65%),-0.0268 (2.02%),-0.0182 (20.40%)
5,13.0,0.0101 (12.69%),0.0006 (95.48%),-0.0065 (77.33%),-0.0271 (35.43%),-0.0352 (32.89%),-0.0738 (19.20%),0.0099 (13.80%),-0.0009 (93.00%),-0.0058 (78.57%),-0.0134 (63.63%),-0.0232 (51.48%),-0.0563 (33.88%)
6,18.0,0.0141 (19.09%),0.0254 (12.36%),0.0475 (8.07%),0.0484 (21.35%),0.0427 (44.99%),0.0555 (39.99%),0.0137 (20.60%),0.0209 (18.80%),0.0335 (19.91%),0.0213 (58.68%),0.0032 (95.55%),0.0104 (87.05%)
7,3.0,-0.0068 (63.44%),-0.0292 (16.28%),-0.0141 (75.85%),-0.0159 (77.28%),0.0376 (43.86%),0.0312 (64.29%),-0.0071 (57.91%),-0.0251 (27.47%),-0.0132 (72.33%),-0.0416 (56.71%),0.0129 (85.10%),-0.0142 (72.17%)
8,54.0,-0.0056 (31.44%),-0.0048 (55.47%),-0.0173 (20.90%),-0.0223 (28.36%),-0.0219 (41.46%),-0.0336 (17.56%),-0.0052 (33.26%),-0.0049 (55.88%),-0.0207 (15.11%),-0.0300 (17.18%),-0.0328 (24.57%),-0.0384 (15.80%)
9,49.0,0.0031 (18.74%),0.0096 (11.65%),0.0015 (88.20%),0.0068 (73.65%),-0.0107 (63.86%),-0.0323 (23.76%),0.0020 (40.03%),0.0035 (55.40%),-0.0097 (29.47%),-0.0079 (68.00%),-0.0281 (21.10%),-0.0560 (4.59%)
10,2.0,-0.0028 (73.52%),-0.0019 (92.60%),-0.0366 (73.24%),-0.1008 (58.25%),-0.1016 (40.51%),-0.1007 (52.93%),0.0006 (89.59%),0.0001 (98.96%),-0.0064 (85.42%),-0.0483 (39.02%),-0.0472 (12.06%),-0.0154 (36.95%)


2016-01-01 - 2017-12-31


,sample size,1d - MKT,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,1d - IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
0,1.0,-0.0032 (nan%),-0.0295 (nan%),0.0076 (nan%),0.0225 (nan%),0.0105 (nan%),-0.0166 (nan%),-0.0009 (nan%),-0.0008 (nan%),0.0101 (nan%),0.0080 (nan%),0.0100 (nan%),0.0051 (nan%)
1,40.0,-0.0072 (55.53%),-0.0073 (75.72%),-0.0169 (59.22%),-0.0358 (41.86%),-0.0256 (59.77%),-0.0117 (82.12%),-0.0099 (38.57%),-0.0026 (90.12%),-0.0041 (87.94%),-0.0083 (82.65%),0.0006 (98.87%),0.0167 (72.04%)
2,22.0,-0.0047 (57.32%),-0.0082 (60.50%),0.0375 (22.35%),0.0309 (41.68%),0.0187 (71.21%),0.0294 (60.80%),-0.0039 (62.28%),0.0007 (96.06%),0.0457 (12.11%),0.0633 (10.06%),0.0683 (18.22%),0.0778 (18.86%)
3,2.0,0.0061 (49.69%),-0.0322 (13.92%),-0.0113 (81.91%),-0.0269 (44.93%),0.0619 (60.35%),0.1154 (68.07%),0.0210 (15.23%),-0.0134 (1.85%),-0.0207 (65.70%),-0.0320 (46.79%),0.0397 (66.06%),0.0461 (79.98%)
4,112.0,0.0013 (60.66%),0.0035 (51.77%),-0.0027 (74.19%),-0.0018 (88.20%),0.0111 (50.46%),0.0058 (80.37%),0.0020 (37.99%),0.0062 (25.41%),0.0005 (94.90%),0.0063 (59.78%),0.0186 (24.55%),0.0187 (41.06%)
5,20.0,0.0040 (27.00%),0.0125 (10.65%),0.0245 (33.78%),-0.0121 (66.46%),-0.0190 (58.59%),-0.0028 (94.10%),0.0054 (16.16%),0.0180 (0.77%),0.0366 (11.53%),0.0030 (91.56%),0.0066 (85.33%),0.0272 (46.35%)
6,13.0,-0.0135 (46.37%),0.0088 (64.74%),0.0062 (83.18%),-0.0200 (57.55%),-0.0608 (17.08%),-0.0403 (37.61%),-0.0146 (46.44%),0.0085 (67.03%),-0.0016 (94.85%),-0.0205 (55.46%),-0.0632 (20.02%),-0.0601 (22.13%)
7,11.0,-0.0108 (15.06%),-0.0121 (58.24%),-0.0249 (29.13%),-0.0477 (11.46%),-0.0302 (51.29%),-0.0493 (30.92%),-0.0099 (11.21%),-0.0104 (61.59%),-0.0315 (17.34%),-0.0338 (31.71%),-0.0230 (62.90%),-0.0228 (62.18%)
8,54.0,0.0020 (67.22%),0.0026 (73.91%),0.0108 (23.95%),0.0117 (42.87%),-0.0103 (59.09%),0.0024 (91.58%),0.0031 (48.69%),0.0044 (51.88%),0.0069 (42.80%),0.0023 (86.48%),-0.0114 (54.10%),0.0027 (89.40%)
9,36.0,-0.0018 (46.37%),-0.0113 (11.73%),-0.0223 (7.43%),-0.0181 (21.83%),-0.0081 (67.98%),0.0056 (81.60%),-0.0030 (25.42%),-0.0131 (7.52%),-0.0243 (6.24%),-0.0290 (8.43%),-0.0332 (12.33%),-0.0332 (18.14%)


2018-01-01 - 2019-12-31


,sample size,1d - MKT,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,1d - IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
1,19.0,-0.0227 (5.41%),-0.0632 (0.19%),-0.0781 (0.82%),-0.1723 (0.42%),-0.2224 (0.06%),-0.2948 (0.04%),-0.0183 (10.40%),-0.0451 (1.41%),-0.0432 (7.98%),-0.1006 (4.40%),-0.1353 (1.12%),-0.1688 (0.84%)
2,22.0,0.0005 (92.45%),-0.0151 (38.64%),-0.0307 (29.52%),-0.0144 (65.36%),-0.0522 (19.26%),-0.0597 (21.98%),0.0004 (93.84%),-0.0164 (29.61%),-0.0437 (10.10%),-0.0389 (16.35%),-0.0878 (4.15%),-0.0799 (11.47%)
3,3.0,-0.0019 (89.64%),0.0371 (45.31%),0.0684 (20.52%),-0.0218 (93.32%),-0.0647 (79.46%),-0.1006 (78.35%),0.0081 (54.09%),0.0563 (21.42%),0.0965 (1.42%),-0.0081 (97.47%),-0.0492 (84.65%),-0.0738 (82.45%)
4,81.0,-0.0068 (26.22%),-0.0036 (69.58%),-0.0102 (52.77%),-0.0402 (2.94%),-0.0653 (0.28%),-0.0767 (0.45%),-0.0064 (27.54%),0.0012 (88.94%),0.0022 (88.19%),-0.0183 (28.90%),-0.0318 (11.88%),-0.0398 (11.71%)
5,17.0,0.0140 (4.73%),0.0228 (22.70%),0.0587 (5.19%),0.0224 (69.01%),-0.0145 (82.15%),-0.0081 (92.65%),0.0115 (8.83%),0.0222 (25.96%),0.0631 (3.29%),0.0306 (56.90%),0.0114 (84.88%),0.0107 (89.37%)
6,4.0,-0.0100 (52.05%),0.0095 (78.73%),0.0143 (88.38%),0.1313 (34.53%),0.1421 (42.31%),0.1279 (51.96%),-0.0082 (67.49%),0.0120 (77.09%),-0.0019 (98.27%),0.0971 (47.95%),0.1246 (48.90%),0.1292 (55.40%)
7,3.0,-0.0262 (34.30%),-0.0207 (38.51%),-0.0502 (50.55%),0.0025 (90.00%),0.0107 (84.20%),-0.0077 (93.20%),-0.0226 (40.53%),-0.0119 (34.04%),-0.0203 (70.57%),0.0229 (30.73%),0.0429 (47.00%),0.0088 (92.90%)
8,34.0,0.0032 (50.90%),-0.0100 (33.00%),-0.0211 (14.73%),-0.0066 (84.27%),-0.0386 (38.76%),-0.0245 (57.99%),0.0021 (64.31%),-0.0152 (13.04%),-0.0387 (0.52%),-0.0238 (48.91%),-0.0420 (33.00%),-0.0371 (37.70%)
9,23.0,-0.0061 (3.18%),-0.0017 (84.93%),-0.0057 (59.64%),-0.0288 (12.00%),-0.0501 (6.43%),-0.0718 (0.79%),-0.0062 (1.55%),-0.0091 (28.01%),-0.0045 (65.29%),-0.0341 (4.53%),-0.0400 (9.45%),-0.0564 (1.03%)
10,4.0,0.0134 (26.01%),0.0324 (46.62%),0.0675 (30.68%),0.0687 (32.82%),-0.0881 (46.49%),-0.1469 (29.05%),0.0128 (11.83%),0.0287 (47.16%),0.0550 (43.26%),0.0469 (54.61%),-0.1106 (29.69%),-0.1610 (23.42%)


In [12]:
import glob

repository_path = f"backtests/LDA/ESG_baseline"
file_path = f"{repository_path}/performance_evaluation_*_*.csv"

master_df = list()
for filename in glob.glob(file_path):
    t0, t1 = filename.split("/")[-1].split(".")[0].split("_")[2:]

    df = pd.read_csv(filename, index_col=0).iloc[-1].to_frame().T
    cols = df.columns.tolist()
    df['t0'], df['t1'] = t0, t1
    df = df[['t0', 't1'] + cols]
    master_df.append(df)
df = pd.concat(master_df).set_index(['t0', 't1']).sort_index()
display(df)  
df.to_csv(f"backtests/LDA/ESG_baseline/9_time_partitions/performance_evaluation_total.csv")
print('\n')

,,sample size,1d - MKT,10d - MKT,30d - MKT,60d - MKT,90d - MKT,120d - MKT,1d - IDT,10d - IDT,30d - IDT,60d - IDT,90d - IDT,120d - IDT
t0,t1,,,,,,,,,,,,,
2000-01-01,2005-12-31,68,0.005 (27.32%),0.0078 (19.3%),0.0187 (11.52%),0.04 (2.25%),0.089 (0.05%),0.113 (0.01%),0.0027 (55.54%),0.0053 (41.86%),0.0155 (21.51%),0.025 (13.89%),0.0687 (0.43%),0.0836 (0.31%)
2006-01-01,2007-12-31,112,0.0054 (18.39%),0.0063 (40.95%),0.016 (12.26%),0.0305 (2.69%),0.0413 (2.88%),0.0634 (0.54%),0.0028 (47.27%),0.0031 (66.32%),0.0049 (60.96%),0.0072 (55.97%),0.0096 (57.69%),0.0183 (35.63%)
2008-01-01,2009-12-31,137,0.0003 (95.54%),0.0033 (70.54%),0.0187 (13.02%),0.0164 (40.57%),0.0201 (41.14%),0.0187 (53.52%),0.0017 (72.79%),0.0042 (57.88%),0.0201 (6.78%),0.0108 (49.62%),0.0091 (64.62%),0.0114 (62.76%)
2010-01-01,2011-12-31,281,0.0018 (25.26%),0.0013 (65.5%),-0.0005 (91.16%),-0.0037 (68.0%),-0.0151 (8.46%),-0.0139 (21.28%),0.0021 (16.4%),0.0005 (86.63%),-0.004 (36.38%),-0.0076 (36.29%),-0.0181 (2.08%),-0.0168 (9.58%)
2012-01-01,2013-12-31,314,-0.0003 (86.91%),0.0052 (14.29%),0.0013 (81.59%),0.0079 (36.04%),0.009 (38.8%),0.0238 (5.26%),0.0006 (70.65%),0.0062 (6.12%),0.004 (46.22%),0.0124 (13.29%),0.0129 (18.21%),0.0256 (2.76%)
2014-01-01,2015-12-31,389,0.0009 (68.69%),-0.0024 (52.12%),-0.0176 (0.26%),-0.0352 (0.0%),-0.0454 (0.0%),-0.0661 (0.0%),0.0005 (81.88%),-0.0032 (38.22%),-0.0176 (0.13%),-0.0317 (0.0%),-0.0401 (0.0%),-0.0544 (0.0%)
2016-01-01,2017-12-31,370,-0.0002 (91.33%),-0.0028 (44.48%),-0.0036 (51.72%),-0.0078 (31.05%),-0.0055 (55.43%),0.0013 (90.93%),-0.0 (98.62%),-0.0001 (97.4%),-0.0023 (64.99%),-0.004 (58.41%),-0.0029 (74.47%),0.0036 (74.5%)
2018-01-01,2019-12-31,250,-0.0033 (17.89%),-0.0052 (25.51%),-0.0102 (18.33%),-0.031 (0.58%),-0.0582 (0.0%),-0.0723 (0.0%),-0.003 (20.9%),-0.0034 (44.09%),-0.0061 (38.42%),-0.0213 (3.95%),-0.0377 (0.23%),-0.0471 (0.12%)
